# model

In [1]:
import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")

In [2]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x, rat):
        x = x + (self.pe[:x.size(0), :]) * rat
        return self.dropout(x)

class TFModule(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, num_hiddens, num_heads, embeddropout_rate, dropout_rate, pos_ratio):
        super(TFModule, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_embed = PositionalEncoding(embed_size, embeddropout_rate, 1025)
        self.pos_ratio = pos_ratio
        
        self.encoder_layer = nn.TransformerEncoderLayer(
                                        d_model=embed_size,
                                        dim_feedforward=num_hiddens,
                                        nhead=num_heads, 
                                        dropout=dropout_rate, 
                                        activation='relu')
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        
        self.decoder = nn.Linear(embed_size, vocab_size) # 初始时间步和最终时间步的隐藏状态作为全连接层输入

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask.to(device)
    
    def forward(self, inputs):
        
        word_emb = self.embedding(inputs.permute(1, 0))
        embeddings = self.pos_embed(word_emb, self.pos_ratio)
#         print(word_emb, '\n', embeddings-word_emb)
        
        # outputs形状是(长度, 批量大小, embeding大小)
        outputs = self.transformer_encoder(embeddings, mask=self._generate_square_subsequent_mask(embeddings.shape[0]))

        output = outputs.permute(1, 0, 2)
        outs = output.reshape(output.size(0)*output.size(1), output.size(2))
        ret = self.decoder(outs)
        return ret

In [3]:
# import torch
# import torch.nn as nn

In [4]:
# encoder_layer = nn.TransformerEncoderLayer(d_model=1, nhead=1, dropout=0, activation='relu')
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

In [5]:
# src = torch.ones(3,1,1)
# out = encoder_layer(src)
# print(src.shape,out.shape)
# print(src)
# print(out)

In [6]:
# # dir(encoder_layer)
# vos = 53
# emb = 22
# numl = 3
# numh = 233
# numhea = 11
# dror = 0
# model = TFModule(vos, emb, numl, numh, numhea, dror)
# src = torch.ones((2,3),dtype=int)
# print(src.shape)
# out = model(src)
# print(out)

# data

In [7]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [8]:
class Corpus(object):
    def __init__(self, train_batch_size=20, eval_batch_size=10, bptt=35):
        self.bptt = bptt
        train_iter = WikiText2(split='train')
        self.tokenizer = get_tokenizer('basic_english')
        counter = Counter()
        for line in train_iter:
            counter.update(self.tokenizer(line))
        self.vocab = Vocab(counter)
        train_iter, val_iter, test_iter = WikiText2()
        train_data = self.data_process(train_iter)
        val_data = self.data_process(val_iter)
        test_data = self.data_process(test_iter)

        self.train_data = self.batchify(train_data, train_batch_size)
        self.val_data = self.batchify(val_data, eval_batch_size)
        self.test_data = self.batchify(test_data, eval_batch_size)

    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],
                           dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

    def batchify(self, data, batch_size):
        # Divide the dataset into batch_size parts.
        nbatch = data.size(0) // batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = data.narrow(0, 0, nbatch * batch_size)
        # Evenly divide the data across the batch_size batches.
        data = data.view(batch_size, -1).t().contiguous()
        return data.to(device)

    def get_batch(self, source, i):
        seq_len = min(self.bptt, len(source) - 1 - i)
        data = source[i:i+seq_len]
        target = source[i+1:i+1+seq_len].reshape(-1)
        return data, target

    def get_ntokens(self):
        return len(self.vocab.stoi)

# main

In [9]:
import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import data
import model
class args():
    pos_ratio = 1
    emsize = 100
    nhid = 200
    nlayers = 3
    nhead = 20
    lr = 0.5
    epochs = 2000
    batch_size = 32
    bptt = 256
    embeddropout = 0.1
    dropout = 0.3
    tied = False
    seed = 1234
    clip = 0.005
    save = 'model.pt'

In [10]:
data_loader = Corpus(train_batch_size=args.batch_size,
                     eval_batch_size=args.batch_size,
                     bptt=args.bptt)

In [11]:
train_data = data_loader.train_data
val_data = data_loader.val_data
test_data = data_loader.test_data

In [12]:
def get_batch(source, i, evaluation=False):
    seq_len = min(args.bptt, len(source) - 1 - i)
#     data = Variable(source[i:i+seq_len], volatile=evaluation)
#     target = Variable(source[i+1:i+1+seq_len].view(-1))
    data = source[i:i+seq_len].t()
    tmp = source[i+1:i+1+seq_len].t()
#     print(data.shape,tmp.shape)
    target = tmp.reshape(-1)
    return data, target

In [13]:
# import torch
# torch.__version__
# print(train_data.size(0))
# for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
#     data, targets = get_batch(train_data, i)
#     print(data.shape,targets.shape,targets[-1])
#     if (batch>=2): break

In [14]:
len(data_loader.vocab.itos)

28783

In [15]:
ntokens = len(data_loader.vocab.itos)
# model = RNNModel(args.model, ntokens, args.emsize, args.nhid,
#         args.nlayers, args.dropout, args.tied)
model = TFModule(ntokens, args.emsize, args.nlayers, args.nhid, args.nhead, args.embeddropout, args.dropout, args.pos_ratio)

In [16]:
model = model.to(device)

In [17]:
model

TFModule(
  (embedding): Embedding(28783, 100)
  (pos_embed): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=200, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=200, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_fe

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

In [19]:
def clip_gradient(optimizer, grad_clip):
    for group in optimizer.param_groups:
        for param in group["params"]:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

def evaluate(data_source):
    model.eval()
    total_loss = 0
    total_words = 0
    for batch, i in enumerate(range(0, data_source.size(0) - 1, args.bptt)):
        data, targets = get_batch(data_source, i)
        
        output = model(data)
        loss = criterion(output, targets)

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
        
#         if batch > 1: break
        
    total_loss = float(total_loss.to(torch.device('cpu')))
#     print(total_loss,total_words)
    return total_loss / total_words, math.exp(total_loss / total_words)

def train():
    model.train()
    total_loss = 0.
    total_words = 0
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        model.zero_grad()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        clip_gradient(optimizer,args.clip)
        optimizer.step()

        total_loss += loss.data * targets.shape[0]
        total_words += targets.shape[0]
        
#         if batch > 10: break
        
#         if (batch % args.log_interval == 0 and batch > 0):
#             cur_loss = total_loss / args.log_interval
#             elapsed = time.time() - start_time
#             print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
#                     'loss {:5.2f} | ppl {:8.2f}'.format(
#                 233, batch, len(train_data) // args.bptt, args.lr,
#                 elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
#             total_loss = 0
#             start_time = time.time()

    cur_loss = total_loss / total_words
    print('train {:10d} words | loss {:5.5f} | ppl {:5.5}'.format(total_words, cur_loss, math.exp(cur_loss)))
    return cur_loss, math.exp(cur_loss)

In [20]:
import numpy as np
Losstrain = []
Lossval = []
Losstest = []

In [ ]:
import time
for T in range(args.epochs):
    print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
    trainl, trainp = train()
    vall, valp = evaluate(val_data)
    testl, testp = evaluate(test_data)
    
    Losstrain.append(trainp)
    Lossval.append(valp)
    Losstest.append(testp)
    np.save('model-2-try-train',np.array(Losstrain))
    np.save('model-2-try-val',np.array(Lossval))
    np.save('model-2-try-test',np.array(Losstest))
    
    print('  valid {:5.5f} | test {:5.5f}'.format(valp, testp))
    print('-'*80)

Round :  0    2021-05-20 19:39:00
train    2049952 words | loss 7.35754 | ppl 1568.0
  valid 798.16332 | test 749.16902
--------------------------------------------------------------------------------
Round :  1    2021-05-20 19:40:44
train    2049952 words | loss 6.81547 | ppl 911.84
  valid 661.89706 | test 623.83616
--------------------------------------------------------------------------------
Round :  2    2021-05-20 19:42:27
train    2049952 words | loss 6.67491 | ppl 792.27
  valid 596.45165 | test 563.12396
--------------------------------------------------------------------------------
Round :  3    2021-05-20 19:44:10
train    2049952 words | loss 6.58580 | ppl 724.73
  valid 554.72322 | test 523.56553
--------------------------------------------------------------------------------
Round :  4    2021-05-20 19:45:54
train    2049952 words | loss 6.51999 | ppl 678.57
  valid 524.46752 | test 494.25424
----------------------------------------------------------------------------

train    2049952 words | loss 5.79643 | ppl 329.12
  valid 285.53222 | test 259.18547
--------------------------------------------------------------------------------
Round :  42    2021-05-20 20:51:20
train    2049952 words | loss 5.78551 | ppl 325.55
  valid 284.25225 | test 258.60280
--------------------------------------------------------------------------------
Round :  43    2021-05-20 20:53:04
train    2049952 words | loss 5.77589 | ppl 322.43
  valid 282.25636 | test 256.65163
--------------------------------------------------------------------------------
Round :  44    2021-05-20 20:54:47
train    2049952 words | loss 5.76499 | ppl 318.94
  valid 280.60940 | test 254.58564
--------------------------------------------------------------------------------
Round :  45    2021-05-20 20:56:30
train    2049952 words | loss 5.75460 | ppl 315.64
  valid 279.57816 | test 253.91329
--------------------------------------------------------------------------------
Round :  46    2021-05-20

train    2049952 words | loss 5.47053 | ppl 237.59
  valid 249.68630 | test 225.65557
--------------------------------------------------------------------------------
Round :  83    2021-05-20 22:01:59
train    2049952 words | loss 5.46525 | ppl 236.34
  valid 250.09925 | test 225.82359
--------------------------------------------------------------------------------
Round :  84    2021-05-20 22:03:42
train    2049952 words | loss 5.45831 | ppl 234.7
  valid 250.57907 | test 225.65265
--------------------------------------------------------------------------------
Round :  85    2021-05-20 22:05:25
train    2049952 words | loss 5.45305 | ppl 233.47
  valid 248.82369 | test 225.59655
--------------------------------------------------------------------------------
Round :  86    2021-05-20 22:07:08
train    2049952 words | loss 5.44664 | ppl 231.98
  valid 249.19802 | test 225.08311
--------------------------------------------------------------------------------
Round :  87    2021-05-20 

train    2049952 words | loss 5.27468 | ppl 195.33
  valid 244.63672 | test 220.70508
--------------------------------------------------------------------------------
Round :  124    2021-05-20 23:12:33
train    2049952 words | loss 5.27106 | ppl 194.62
  valid 243.66805 | test 220.93611
--------------------------------------------------------------------------------
Round :  125    2021-05-20 23:14:16
train    2049952 words | loss 5.26665 | ppl 193.77
  valid 246.81406 | test 222.97058
--------------------------------------------------------------------------------
Round :  126    2021-05-20 23:16:00
train    2049952 words | loss 5.26400 | ppl 193.25
  valid 245.28492 | test 221.99536
--------------------------------------------------------------------------------
Round :  127    2021-05-20 23:17:43
train    2049952 words | loss 5.26031 | ppl 192.54
  valid 243.90872 | test 220.35968
--------------------------------------------------------------------------------
Round :  128    2021-

train    2049952 words | loss 5.14490 | ppl 171.55
  valid 243.22983 | test 221.10359
--------------------------------------------------------------------------------
Round :  165    2021-05-21 00:23:09
train    2049952 words | loss 5.14199 | ppl 171.06
  valid 244.73660 | test 222.32793
--------------------------------------------------------------------------------
Round :  166    2021-05-21 00:24:52
train    2049952 words | loss 5.13924 | ppl 170.59
  valid 242.88798 | test 221.62548
--------------------------------------------------------------------------------
Round :  167    2021-05-21 00:26:36
train    2049952 words | loss 5.13671 | ppl 170.15
  valid 245.54766 | test 223.42260
--------------------------------------------------------------------------------
Round :  168    2021-05-21 00:28:19
train    2049952 words | loss 5.13538 | ppl 169.93
  valid 245.68258 | test 223.54066
--------------------------------------------------------------------------------
Round :  169    2021-

...

  valid 243.78018 | test 227.17644
--------------------------------------------------------------------------------
Round :  1312    2021-05-22 07:00:40
train    2049952 words | loss 4.57948 | ppl 97.463
  valid 241.31026 | test 224.40804
--------------------------------------------------------------------------------
Round :  1313    2021-05-22 07:02:14
train    2049952 words | loss 4.57960 | ppl 97.476
  valid 241.61749 | test 225.31162
--------------------------------------------------------------------------------
Round :  1314    2021-05-22 07:03:47
train    2049952 words | loss 4.58041 | ppl 97.554
  valid 238.33278 | test 222.25002
--------------------------------------------------------------------------------
Round :  1315    2021-05-22 07:05:20
train    2049952 words | loss 4.58071 | ppl 97.583
  valid 242.93146 | test 225.96862
--------------------------------------------------------------------------------
Round :  1316    2021-05-22 07:06:53
train    2049952 words | loss 4

train    2049952 words | loss 4.57374 | ppl 96.906
  valid 239.68507 | test 223.59151
--------------------------------------------------------------------------------
Round :  1353    2021-05-22 08:04:20
train    2049952 words | loss 4.57477 | ppl 97.006
  valid 239.91621 | test 223.84569
--------------------------------------------------------------------------------
Round :  1354    2021-05-22 08:05:53
train    2049952 words | loss 4.57377 | ppl 96.908
  valid 240.45107 | test 223.99975
--------------------------------------------------------------------------------
Round :  1355    2021-05-22 08:07:26
train    2049952 words | loss 4.57451 | ppl 96.981
  valid 240.71664 | test 224.67511
--------------------------------------------------------------------------------
Round :  1356    2021-05-22 08:09:00
train    2049952 words | loss 4.57349 | ppl 96.881
  valid 238.77583 | test 222.33976
--------------------------------------------------------------------------------
Round :  1357    

  valid 241.31435 | test 224.73783
--------------------------------------------------------------------------------
Round :  1393    2021-05-22 09:06:25
train    2049952 words | loss 4.56721 | ppl 96.275
  valid 242.81632 | test 226.03182
--------------------------------------------------------------------------------
Round :  1394    2021-05-22 09:07:58
train    2049952 words | loss 4.56739 | ppl 96.293
  valid 240.15233 | test 223.74169
--------------------------------------------------------------------------------
Round :  1395    2021-05-22 09:09:31
train    2049952 words | loss 4.56744 | ppl 96.297
  valid 241.35037 | test 224.28140
--------------------------------------------------------------------------------
Round :  1396    2021-05-22 09:11:04
train    2049952 words | loss 4.56697 | ppl 96.252
  valid 241.17803 | test 224.91343
--------------------------------------------------------------------------------
Round :  1397    2021-05-22 09:12:38
train    2049952 words | loss 4

train    2049952 words | loss 4.56152 | ppl 95.729
  valid 241.49115 | test 224.96715
--------------------------------------------------------------------------------
Round :  1434    2021-05-22 10:10:07
train    2049952 words | loss 4.56078 | ppl 95.658
  valid 241.55029 | test 224.38843
--------------------------------------------------------------------------------
Round :  1435    2021-05-22 10:11:40
train    2049952 words | loss 4.56233 | ppl 95.807
  valid 239.97735 | test 224.03680
--------------------------------------------------------------------------------
Round :  1436    2021-05-22 10:13:13
train    2049952 words | loss 4.56004 | ppl 95.587
  valid 240.32183 | test 224.09031
--------------------------------------------------------------------------------
Round :  1437    2021-05-22 10:14:47
train    2049952 words | loss 4.56123 | ppl 95.701
  valid 241.22107 | test 224.77315
--------------------------------------------------------------------------------
Round :  1438    

  valid 239.41381 | test 222.93992
--------------------------------------------------------------------------------
Round :  1474    2021-05-22 11:12:11
train    2049952 words | loss 4.55586 | ppl 95.188
  valid 240.41728 | test 223.50046
--------------------------------------------------------------------------------
Round :  1475    2021-05-22 11:13:43
train    2049952 words | loss 4.55523 | ppl 95.129
  valid 240.08569 | test 223.24136
--------------------------------------------------------------------------------
Round :  1476    2021-05-22 11:15:15
train    2049952 words | loss 4.55540 | ppl 95.145
  valid 240.81126 | test 223.94175
--------------------------------------------------------------------------------
Round :  1477    2021-05-22 11:16:46
train    2049952 words | loss 4.55548 | ppl 95.153
  valid 240.85662 | test 223.36983
--------------------------------------------------------------------------------
Round :  1478    2021-05-22 11:18:19
train    2049952 words | loss 4

In [ ]:
# torch.save(model.state_dict(), 'parameter-2.pkl')
# evaluate(test_data)

In [ ]:
# if 1:
#     x = torch.ones((1,1),dtype=int)
#     x[0,0] = 1
#     print(model.embedding(x))

In [ ]:
# import time
# for T in range(100):
#     print('Round : ',T,"  ",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
#     train()
#     print('  valid ',evaluate(val_data))
#     print('  test  ',evaluate(test_data))
#     print('-'*80)

In [ ]:
# if 1:
#     x = torch.tensor([[1000,1],[100,1],[1000,1]],dtype=float)
#     y = torch.tensor([0,1,0])
#     print(criterion(x,y))

In [ ]:
# -math.log(0.1)